In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re

encoding = "ISO-8859-1"
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/eugene/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/eugene/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/eugene/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))


def replaces(s):
    if isinstance(s, str):
        s = s.lower()
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) 
    
        s = s.replace(" x "," xby ")
        s = s.replace("*"," xby ")
        s = s.replace(" by "," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
        s = s.replace("0x","0 xby ")
        s = s.replace("1x","1 xby ")
        s = s.replace("2x","2 xby ")
        s = s.replace("3x","3 xby ")
        s = s.replace("4x","4 xby ")
        s = s.replace("5x","5 xby ")
        s = s.replace("6x","6 xby ")
        s = s.replace("7x","7 xby ")
        s = s.replace("8x","8 xby ")
        s = s.replace("9x","9 xby ")
        
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)
        
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")

        s = s.replace("  "," ")
        
        s = s.replace("/", " ")
        s = s.replace("-", " ")
        return " ".join([re.sub('[^A-Za-z0-9./]', ' ', word)
                         for word in s.split()])
    else:
        return "null"

@np.vectorize
def stem_sentence(s): 
    s = replaces(s)   
    return str(" ".join([stemmer.stem(word)
                         for word in s.split()
                         if word not in stop_words]))


In [3]:
@np.vectorize
def word_match_count(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        count += 1 if text.count(word) > 0 else 0
    return count / len(search_term)

In [4]:
products = pd.read_csv('descr.csv', index_col=['product_uid'])
products['orig_descr'] = products['orig_descr'].astype(str)
products['descr'] = products['descr'].astype(str)
#products = pd.read_csv('product_descriptions.csv', encoding=encoding, index_col=['product_uid'])
#products['orig_descr'] = products['product_description'].astype(str)
#products['descr'] = stem_sentence(products['product_description'].astype(str))

In [5]:
brand = pd.read_csv('brand.csv', index_col=['product_uid'])
brand['value'] = brand['value'].astype(str)
#attrs = pd.read_csv('attributes.csv')
#brand = attrs[attrs['name'] == 'MFG Brand Name']
#brand.index = brand['product_uid']
#brand['value'] = stem_sentence(brand['value'].astype(str))
#brand.head()

In [ ]:
#brand.to_csv('brand.csv')
#products.to_csv('descr.csv')

In [6]:
@np.vectorize
def word_match_std(search_term, text):
    indexes = []
    search_term = search_term.split()
    text = text.split()    
    n = len(text)
    if len(search_term) == 0:
        return 0
    for word in set(search_term):
        if word in text:
            indexes.append(text.index(word))
    if len(indexes) > 0:
        return np.var(indexes)
    else:
        return 0

@np.vectorize
def match_last_word(search_term, text):
    last_word = search_term.split()[-1]
    if text.find(last_word) > -1:
        return 1
    else:
        return 0
    
@np.vectorize    
def word_match_index(search_term, text):
    count = 0
    search_term = search_term.split()
    text = text.split()
    if len(search_term) == 0:
        return 0
    min_ind = len(text)
    for word in set(search_term):
        if word in text:
            min_ind = min(min_ind, text.index(word))
    return min_ind / len(text) 

@np.vectorize 
def match_word_n_pos(text, search_term):
    text = nltk.pos_tag(replaces(text).split())
    search_pos = nltk.pos_tag(replaces(search_term).split())
    text = ["%s/%s" % (word[0].lower(), word[1]) for word in text]
    search_pos = ["%s/%s" % (word[0].lower(), word[1]) for word in search_pos]
    
    if len(search_pos) == 0:
        return 0
    matches = 0
    for word in set(text):
        if word in search_pos:
            matches += 1
    return matches / len(search_pos)

In [10]:
train = pd.read_csv('train.csv', encoding=encoding, index_col=['id'])
train['orig_search_term'] = train['search_term'].astype(str)
train['orig_product_title'] = train['product_title'].astype(str)
train['descr'] = train.product_uid.map(products['descr'])
train['orig_descr'] = train.product_uid.map(products['orig_descr'])
for col in ['product_title', 'search_term']:
    train[col] = stem_sentence(train[col].astype(str))

train.loc[train['search_term'] == '', 'search_term'] = 'null'

for col in ['product_title', 'descr']:
    train['match_' + col] = word_match_count(train['search_term'], train[col])

train['len_search_term_words'] = train['search_term'].apply(lambda x: len(x.split()))
train['len_search_term_letters'] = train['search_term'].str.len()
train['complete_match_title'] = \
    train.apply(lambda row:1 if row['product_title'].find(row['search_term']) >= 0 else 0, axis=1)   
train['complete_match_descr'] = \
    train.apply(lambda row:1 if row['descr'].find(row['search_term']) >= 0 else 0, axis=1)

train['brand'] = train['product_uid'].map(brand['value']).fillna('')
train['match_brand'] = word_match_count(train['brand'], train['search_term'])
train['words_std_title'] = word_match_std(train['search_term'], train['product_title'])
train['words_std_descr'] = word_match_std(train['search_term'], train['descr'])

train['last_word_title'] = match_last_word(train['search_term'], train['product_title'])  
train['last_word_descr'] = match_last_word(train['search_term'], train['descr']) 

train['prod_title_ind'] = train.apply(lambda x: word_match_index(x['search_term'], x['product_title']), axis=1)
train['descr_ind'] = train.apply(lambda x: word_match_index(x['search_term'], x['descr']), axis=1)

train['match_pos_title'] = match_word_n_pos(train['orig_product_title'], train['orig_search_term'])
train['match_pos_descr'] = match_word_n_pos(train['orig_descr'], train['orig_search_term'])

train['match_orig_title'] = word_match_count(train['search_term'], train['orig_product_title'])
train['match_orig_descr'] = word_match_count(train['search_term'], train['orig_descr'])
train['len_title'] = train['product_title'].apply(lambda x: len(x.split()))
train['len_descr'] = train['descr'].apply(lambda x: len(x.split()))

In [13]:
tree_cols = ['match_product_title', 'match_descr', 'len_search_term_words', 'len_search_term_letters',
             'match_pos_title', 'match_brand', 'match_pos_descr', 'prod_title_ind', 'descr_ind',
             'words_std_title', 'words_std_descr', 'len_title', 'len_descr', 'last_word_title',
             'last_word_descr', 'match_orig_title', 'match_orig_descr']

use_cols = tree_cols + ['search_term', 'product_title', 'descr', 'brand']
train[use_cols].head()

,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_brand,match_pos_descr,prod_title_ind,descr_ind,words_std_title,...,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,search_term,product_title,descr,brand
id,,,,,,,,,,,,,,,,,,,,,
2,0.500000,0.500000,2,12,0.500000,0.0,0.000000,0.8333333333333334,0.011235955056179775,0.00,...,6,89,0,0,0.0,0.0,angl bracket,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
3,0.000000,0.000000,2,9,0.000000,0.0,0.000000,1.0,1.0,0.00,...,6,89,0,0,0.0,0.0,l bracket,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
9,0.000000,0.000000,1,4,0.000000,0.0,0.500000,1.0,1.0,0.00,...,12,122,1,1,0.0,0.0,deck,behr premium textur deckov 1 gal. sc 141 tugbo...,behr premium textur deckov innov solid color c...,behr premium textur deckov
16,0.333333,0.333333,3,16,0.333333,0.0,0.000000,0.36363636363636365,0.07352941176470588,0.00,...,11,68,0,0,0.0,0.0,rain shower head,delta vero 1 handl shower faucet trim kit chro...,updat bathroom delta vero single-handl shower ...,delta
17,1.000000,1.000000,2,13,0.666667,0.0,0.333333,0.36363636363636365,0.07352941176470588,0.25,...,11,68,1,1,0.0,0.0,shower faucet,delta vero 1 handl shower faucet trim kit chro...,updat bathroom delta vero single-handl shower ...,delta


In [14]:
train.to_csv('train2.csv')

In [16]:
test = pd.read_csv('test.csv', encoding=encoding, index_col=['id'])
test['orig_search_term'] = test['search_term']
test['orig_product_title'] = test['product_title']
test['descr'] = test.product_uid.map(products['descr'])
test['orig_descr'] = test.product_uid.map(products['orig_descr'])
for col in ['product_title', 'search_term']:
    test[col] = stem_sentence(test[col])

test.loc[test['search_term'] == '', 'search_term'] = 'null'

for col in ['product_title', 'descr']:
    test['match_' + col] = word_match_count(test['search_term'], test[col])

test['len_search_term_words'] = test['search_term'].apply(lambda x: len(x.split()))
test['len_search_term_letters'] = test['search_term'].str.len()
test['complete_match_title'] = \
    test.apply(lambda row:1 if row['product_title'].find(row['search_term']) >= 0 else 0, axis=1)   
test['complete_match_descr'] = \
    test.apply(lambda row:1 if row['descr'].find(row['search_term']) >= 0 else 0, axis=1)

test['brand'] = test['product_uid'].map(brand['value']).fillna('')
test['search_term'] = test['search_term'].astype(str)
test['match_brand'] = word_match_count(test['brand'], test['search_term'])
test['words_std_title'] = word_match_std(test['search_term'], test['product_title'])
test['words_std_descr'] = word_match_std(test['search_term'], test['descr'])

test['last_word_title'] = match_last_word(test['search_term'], test['product_title'])  
test['last_word_descr'] = match_last_word(test['search_term'], test['descr']) 


test['match_pos_title'] = match_word_n_pos(test['orig_product_title'], test['orig_search_term'])
test['match_pos_descr'] = match_word_n_pos(test['orig_descr'], test['orig_search_term'])

test['match_orig_title'] = word_match_count(test['search_term'], test['orig_product_title'])
test['match_orig_descr'] = word_match_count(test['search_term'], test['orig_descr'])
test['len_title'] = test['product_title'].apply(lambda x: len(x.split()))
test['len_descr'] = test['descr'].apply(lambda x: len(x.split()))
test['prod_title_ind'] = word_match_index(test['search_term'], test['product_title'])
test['descr_ind'] = word_match_index(test['search_term'], test['descr'])

In [17]:
test[use_cols].head()

,match_product_title,match_descr,len_search_term_words,len_search_term_letters,match_pos_title,match_brand,match_pos_descr,prod_title_ind,descr_ind,words_std_title,...,len_title,len_descr,last_word_title,last_word_descr,match_orig_title,match_orig_descr,search_term,product_title,descr,brand
id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,2,14,0.000000,0.0,0.000000,1.0,1.00000,0,...,6,89,0,0,0.0,0.0,90deg. bracket,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
4,0.000000,0.000000,3,15,0.000000,0.0,0.000000,1.0,1.00000,0,...,6,89,0,0,0.0,0.0,metal l bracket,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
5,0.333333,0.333333,3,15,0.333333,0.5,0.333333,0.0,0.11236,0,...,6,89,0,0,0.0,0.0,simpson sku abl,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
6,1.000000,0.333333,3,18,0.666667,0.5,0.666667,0.0,0.11236,0,...,6,89,1,0,0.0,0.0,simpson strong tie,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti
7,0.750000,0.250000,4,25,0.500000,0.5,0.750000,0.0,0.11236,0,...,6,89,0,0,0.0,0.0,simpson strong tie hcc668,simpson strong tie 12 gaug angl,"not angl make joint stronger, also provid cons...",simpson strong-ti


In [18]:
test.to_csv('test2.csv')

In [19]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion

In [20]:
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, train):
        return train[tree_cols].values


class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, x, y=None):
        return self
    def transform(self, dataset):
        return dataset[self.col].apply(str)

In [22]:
clf = Pipeline([
    ('union', FeatureUnion(
                transformer_list = [
                    ('cst',  cust_regression_vals()),  
                    ('txt1', Pipeline([('s1', cust_txt_col(col='search_term')),
                                       ('tfidf1', TfidfVectorizer(stop_words='english')),
                                       ('tsvd1', TruncatedSVD(n_components=80))])),
                    ('txt2', Pipeline([('s2', cust_txt_col(col='product_title')),
                                       ('tfidf2', TfidfVectorizer(stop_words='english')),
                                       ('tsvd2', TruncatedSVD(n_components=80))])),
                    ('txt3', Pipeline([('s3', cust_txt_col(col='descr')),
                                       ('tfidf3', TfidfVectorizer(stop_words='english')),
                                       ('tsvd3', TruncatedSVD(n_components=60))])),
                    ('txt4', Pipeline([('s4', cust_txt_col(col='brand')),
                                       ('tfidf4', TfidfVectorizer(stop_words='english')),
                                       ('tsvd4', TruncatedSVD(n_components=30))]))
                    ]
            ))])
clf.fit(train[use_cols])
X_train = clf.transform(train[use_cols])
X_test = clf.transform(test[use_cols])

In [23]:
y = train['relevance'].astype(np.float).values

gbm = lgb.LGBMRegressor(num_leaves=70, colsample_bytree=0.5)
params = {
    'learning_rate': [0.02, 0.03, 0.05],
    'n_estimators': [400, 500, 700]
}
model = GridSearchCV(gbm, params,
                     cv=4,
                     n_jobs=1,
                     return_train_score=False,
                     verbose=2)
model.fit(X_train, y)
grid_df = pd.DataFrame(model.cv_results_)
grid_df.drop(columns=grid_df.columns[:3], inplace=True)
grid_df.sort_values(['rank_test_score'], inplace=True)
grid_df.head()

Fitting 4 folds for each of 9 candidates, totalling 36 fits
[CV] learning_rate=0.02, n_estimators=400 ............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............. learning_rate=0.02, n_estimators=400, total=  17.2s
[CV] learning_rate=0.02, n_estimators=400 ............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.3s remaining:    0.0s


[CV] ............. learning_rate=0.02, n_estimators=400, total=  15.9s
[CV] learning_rate=0.02, n_estimators=400 ............................
[CV] ............. learning_rate=0.02, n_estimators=400, total=  15.1s
[CV] learning_rate=0.02, n_estimators=400 ............................
[CV] ............. learning_rate=0.02, n_estimators=400, total=  14.7s
[CV] learning_rate=0.02, n_estimators=500 ............................
[CV] ............. learning_rate=0.02, n_estimators=500, total=  18.3s
[CV] learning_rate=0.02, n_estimators=500 ............................
[CV] ............. learning_rate=0.02, n_estimators=500, total=  13.9s
[CV] learning_rate=0.02, n_estimators=500 ............................
[CV] ............. learning_rate=0.02, n_estimators=500, total=  21.1s
[CV] learning_rate=0.02, n_estimators=500 ............................
[CV] ............. learning_rate=0.02, n_estimators=500, total=  16.9s
[CV] learning_rate=0.02, n_estimators=700 ............................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 10.2min finished


,std_score_time,param_learning_rate,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
5,0.008243,0.03,700,"{'learning_rate': 0.03, 'n_estimators': 700}",0.257159,0.272576,0.285935,0.134840,0.237629,0.060211,1
8,0.027176,0.05,700,"{'learning_rate': 0.05, 'n_estimators': 700}",0.252267,0.271938,0.284974,0.135866,0.236262,0.059120,2
7,0.048576,0.05,500,"{'learning_rate': 0.05, 'n_estimators': 500}",0.252104,0.268807,0.283535,0.132533,0.234246,0.059766,3
4,0.008431,0.03,500,"{'learning_rate': 0.03, 'n_estimators': 500}",0.253400,0.266111,0.280504,0.130431,0.232613,0.059768,4
6,0.014043,0.05,400,"{'learning_rate': 0.05, 'n_estimators': 400}",0.250997,0.267027,0.281359,0.129188,0.232144,0.060403,5


In [24]:
y_pred = model.predict(X_test)
y_pred[y_pred>3.] = 3.
y_pred[y_pred<1.] = 1.
results = pd.DataFrame({'id':test.index.values, 'relevance':y_pred})
results.to_csv('lgbm4.csv', header=True, index=False)

0.47135

In [26]:
train['pred_diff'] = (train['relevance'] - model.predict(X_train))
pd.set_option('display.max_colwidth', -1)
train.sort_values(['pred_diff'], ascending=False)

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
203914,192088,behr premium plus 600e 2 harbor mist zero voc interior paint,locit 2 plus 1,3.00,locite 2 plus 1,BEHR Premium Plus #600E-2 Harbor Mist Zero VOC Interior Paint,"behr premium plus zero voc, self-prim interior flat 100% acryl provid mildew resist finish. this easy-clean flat provid excel touch-up, ultim durabl except hide. the finish display non-reflective, matt appearance.california residents: see&nbsp;proposit 65 informationid famili rooms, live rooms, dine rooms, bedroom ceilings100% acryl finish easi cleaningtough durabl matt finishexcept hide, 1 cover 400 sq. ft. depend color select surfac porosityeasi soap water clean-upexcept latex formula provid mildew resist finishself prime finish seal surfaceactu paint color may vari on-screen printer representationsonlin price includ paint care fee follow states: ca, co, ct, me, mn, or, ri, vt","BEHR Premium Plus Zero VOC, Self-Priming Interior Flat is 100% acrylic and provides a mildew resistant finish. This easy-clean flat provides excellent touch-up, ultimate durability and exceptional hide. The finish displays a non-reflective, matte appearance.California residents: see&nbsp;Proposition 65 informationIdeal for family rooms, living rooms, dining rooms, bedrooms and ceilings100% acrylic finish for easy cleaningTough and durable matte finishExceptional hide, 1 can covers up to 400 sq. ft. depending on color selected and surface porosityEasy soap and water clean-upExceptional latex formula provides a mildew resistant finishSelf priming finish that seals the surfaceActual paint colors may vary from on-screen and printer representationsOnline Price includes Paint Care fee in the following states: CA, CO, CT, ME, MN, OR, RI, VT",0.500000,0.500000,...,1,0.18181818181818182,0.021052631578947368,0.500000,0.500000,0.000000,0.250000,11,95,1.162489
194147,184559,lenmar nickel metal hydrid 1200mah 3.6 volt cordless phone replac batteri,polish batteri metal,3.00,polisher battery metal,Lenmar Nickel-Metal Hydride 1200mAh/3.6-Volt Cordless Phone Replacement Battery,"power panasonic, american telecom, tele-phone, key phone, audiovox, uniden, universal, zeta cordless phone lenmar nickel-met hydrid cordless phone replac battery. this 3.6-volt/1200mah batteri provid reliable, long-last power. compat panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510.fits: panason kx-a150, american telecom 2110, tele-phon tel-1000, key phone 920-wd, audiovox at-12a, uniden ex-3300, univers 2000, zeta 510replaces: dantona 3aa, empir cpb-400, general electr bt-17, panason p-p341pa, recoton t121, uniden ba-3003.6-volt / 1200mahnickel-met hydride1 year warranti","Power your Panasonic, American Telecom, Tele-Phone, Key Phone, Audiovox, Uniden, Universal, and Zeta cordless phones with Lenmar's Nickel-Metal Hydride cordless phone replacement battery. This 3.6-Volt/1200mAh battery will provide reliable, long-lasting power. Compatible with Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, and Zeta 510.Fits: Panasonic KX-A150, American Telecom 2110, Tele-Phone TEL-1000, Key Phone 920-WD, Audiovox AT-12A, Uniden EX-3300, Universal 2000, Zeta 510Replaces: Dantona 3AA, Empire CPB-400, General Electric BT-17, Panasonic P-P341PA, Recoton T121, Uniden BA-3003.6-volt / 1200mAhNickel-metal hydride1 year warranty",0.666667,0.333333,...,0,0.18181818181818182,0.2682926829268293,0.666667,0.666667,0.000000,0.000000,11,82,1.120851
193001,183692,lichtenberg white no. 918 millenni ryan heather textur sheer curtain panel 40in. w xbi 95in. l,w g 918,3.00,w g 918,"LICHTENBERG Whit

In [27]:
train.sort_values(['pred_diff'], ascending=True)

,product_uid,product_title,search_term,relevance,orig_search_term,orig_product_title,descr,orig_descr,match_product_title,match_descr,...,last_word_descr,prod_title_ind,descr_ind,match_pos_title,match_pos_descr,match_orig_title,match_orig_descr,len_title,len_descr,pred_diff
id,,,,,,,,,,,,,,,,,,,,,
136808,148897,earthquak 212cc tiller rear tine crt side shield,side shield,1.00,side shields,Earthquake 212cc Tiller Rear Tine CRT with Side Shields,"the earthquak 6015v rear tine rototil deliv ultim combin power size. rear-mount tine till stubborn dirt easili high-perform 212cc viper engine. we engin compact rear tine rototil power larger counterparts.easi - single-hand control sodearthquak 6015v rear tinecompact - full-siz power compact framedur - cast iron, bronz gear drive transmiss","The Earthquake 6015V rear tine rototiller delivers the ultimate combination of power and size. Rear-mounted tines till stubborn dirt easily with a high-performance 212cc Viper engine. We have engineered a compact rear tine rototiller with the same power as its larger counterparts.Easy - single-handed control through sodEarthquake 6015V rear tineCompact - full-sized power in a compact frameDurable - cast iron, bronze gear drive transmission",1.000000,0.000000,...,0,0.75,1.0,1.000000,0.000000,0.000000,0.000000,8,49,-1.559468
69679,119478,romano 4ft. boxwood spiral topiari tree,topiari tree,1.00,topiary tree,Romano 4 ft. Boxwood Spiral Topiary Tree,"enhanc home romano boxwood spiral topiari tree. this wonder full bodi boxwood spiral topiari tree keep color rest lose color leaves. you alway look forward rich green tone romano boxwood spiral topiari tree home. if look high qualiti artifici tree make bold statement home, it.light mobil designov 700 leavesindoor/outdooriron frameuv resist rubber leaves7 in. potenh room hous","Enhance your home with a Romano Boxwood Spiral Topiary Tree. This wonderful full bodied Boxwood Spiral Topiary Tree keeps its colors when all the rest lose their color and leaves. You can always look forward to the rich green tones of the Romano Boxwood Spiral Topiary Tree in your home. If you are looking for a High quality artificial tree to make a bold statement in your home, this is it.Light and mobile designOver 700 leavesIndoor/outdoorIron frameUV resistant rubber leaves7 in. potEnhances any room in your house",1.000000,1.000000,...,1,0.6666666666666666,0.08771929824561403,1.000000,1.000000,0.000000,0.500000,6,57,-1.516477
85851,125582,oakland live 26in. metal grape tabl plant stand,grape plant,1.00,grape plant,Oakland Living 26 in. Metal Grape Table Plant Stand,"the oakland live 26 in. metal grape tabl plant stand offer sturdi beauti place keep plant display. this stand made cast iron durabl antiqu bronz powder coat finish.mad cast ironantiqu bronz harden powder coat finish ad durabilityresist fading, chip crackingeasi follow assembl instruct hardwar includedstainless steel brass assembl hardwarefade, chip crack resistantsom assembl required1 year limit manufactur warrantyoth item avail collect","The Oakland Living 26 in. Metal Grape Table Plant Stand offers a sturdy and beautiful place to keep plants on display. This stand is made from cast iron for durability with an antique bronze powder coat finish.Made from cast ironAntique bronze hardened powder coat finish for added durabilityResistant to fading, chipping or crackingEasy to follow assembly instructions and hardware includedStainless steel or brass assembly hardwareFade, chip and crack resistantSome assembly required1 year limited manufacturer's warrantyOther items available in collection",1.000000,1.000000,...,1,0.5,0.09836065573770492,1.000000,1.000000,0.000000,0.000000,8,61,-1.486813
120624,140844,rachael ray 10 qt. cover stockpot,rachael ray,1.00,rachael ray,Rachael Ray 10 qt. Covered Stockpot,"whether boil pasta, make batch chili cook grains, large, 10 quart stockpot right size. when stovetop, two short side handl help take less space cabinet. the hard-anod construct 